# Nested sampling

In the [MCMC tutorial](./mcmc.html), Bayes theorem was shown, 

$$ p = \frac{P L}{Z}, $$

where where, $p$ is the posterior probability, $P$ is the prior probability, $L$ is the likelihood, and $Z$ is the evidence. 
Normally, in the evaluation of the posterior probability, the evidence is removed as a constant of propertionality. 
This is due to the fact that it is a single value for a given model and dataset pair. 

However, sometimes it is desirable to find the evidence for a model to a given dataset.
In particular, when we want to compare the evidence for a series of models, to determine which best describes the dataset.
In order to achieve this, `uravu` is able to perform [nested sampling](https://doi.org/10.1063/1.1835238) to estimate the Bayesian evidence for a model given a particular dataset. 

This tutorial will show how this is achieved, and show the utility of Bayesian model selection. 

However, as always we must first create some *synthetic* data. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
np.random.seed(2)

In [ ]:
x = np.linspace(10, 50, 20)
y = .3 * x ** 2 - 1.4 * x + .2
y += y * np.random.randn(20) * 0.05
dy = 3 * x

In [ ]:
plt.errorbar(x, y, dy, marker='o', ls='')
plt.show()

From looking at the code used to synthesize this data, it is clear that the functional from of the model is a quadratic polynomial. 
However, if this was data collected from some measurement and the physical theory suggests it should be analysed with a polynormial of unknown degree then Bayesian model selection would be the ideal tool to find the best model. 

Let's quickly write a few models to perform the `n` degree polynomial analysis with. 

In [ ]:
def one_degree(x, a, b):
    return b * x + a

def two_degree(x, a, b, c):
    return c * x ** 2 + b * x + a

def three_degree(x, a, b, c, d):
    return d * x ** 3 + c * x ** 2 + b * x + a

def four_degree(x, a, b, c, d, e):
    return e * x ** 4 + d * x ** 3 + c * x ** 2 + b * x + a

With these functions defined, we can now build the `Relationship` objects for each of the functions. 

In [ ]:
from uravu.relationship import Relationship

In [ ]:
one_modeller = Relationship(one_degree, x, y, dy)
one_modeller.max_likelihood()

In [ ]:
two_modeller = Relationship(two_degree, x, y, dy)
two_modeller.max_likelihood()

In [ ]:
three_modeller = Relationship(three_degree, x, y, dy)
three_modeller.max_likelihood()

In [ ]:
four_modeller = Relationship(four_degree, x, y, dy)
four_modeller.max_likelihood()

Having built these, lets see what the maximum likelihood variables are for each. 

In [ ]:
print(one_modeller.variables)

In [ ]:
print(two_modeller.variables)

In [ ]:
print(three_modeller.variables)

In [ ]:
print(four_modeller.variables)

We can see that the highest order terms are quite small for the larger degree of polynomial. 
Let's see what effect this has on the evidence. 

Note that `uravu` and `dynesty` calculates the natural log of the evidence, $\ln{Z}$. 

In [ ]:
one_modeller.nested_sampling()

In [ ]:
two_modeller.nested_sampling()

In [ ]:
three_modeller.nested_sampling()

In [ ]:
four_modeller.nested_sampling()

Having estimated $\ln{Z}$ for each `Relationship`, lets plot them as a function of the number of variables in the model.

In [ ]:
variables = [len(one_modeller.variables), len(two_modeller.variables),
             len(three_modeller.variables), len(four_modeller.variables)]
ln_evidence = [one_modeller.ln_evidence.n, two_modeller.ln_evidence.n,
               three_modeller.ln_evidence.n, four_modeller.ln_evidence.n]
ln_evidence_err = [one_modeller.ln_evidence.s, two_modeller.ln_evidence.s,
                   three_modeller.ln_evidence.s, four_modeller.ln_evidence.s]

In [ ]:
plt.errorbar(variables, ln_evidence, ln_evidence_err, marker='o', ls='')
plt.xlabel('Number of variables')
plt.ylabel(r'$\ln{Z}$')
plt.show()

We can see that the evidence reaches a maxiumum at 3 variables (the `two_degree` model), this indicates that this is the most probable model for analysis of this dataset. 

Finally, we can use some build in functionality of `uravu` to compare different evidence values, in a value known as the [Bayes factor](https://doi.org/10.2307/2291091). 
Let's compare the `one_degree` and `two_degree` models.

In [ ]:
from uravu import utils

In [ ]:
print(utils.bayes_factor(two_modeller.ln_evidence, one_modeller.ln_evidence))

The Bayes factor between the two and one degree models, $2\ln{B_{21}}$, has a value of ~9. 
The Table in [Kass and Raftery](https://doi.org/10.2307/2291091) suggests that this shows "Strong" evidence for the model with more variables (the `two_degree` model). 

This means that it is sensible to go ahead and use the `two_degree` model in the further analysis of our data, using MCMC for example. 

In [ ]:
two_modeller.mcmc()

In [ ]:
from uravu import plotting

In [ ]:
plotting.plot_relationship(two_modeller)
plt.show()